In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
#         print(os.path.join(dirname, filename))
          pass

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Tensorflow Input Pipeline

In [2]:
import tensorflow as tf

2024-07-25 01:50:30.363531: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-25 01:50:30.363705: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-25 01:50:30.528941: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Create tf dataset from a list

In [3]:
daily_sales_numbers = [21, 22, -108, 31, -1, 32, 34,31]

tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)
tf_dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

# Iterate through tf dataset

In [4]:
for sales in tf_dataset:
    print(sales.numpy())

21
22
-108
31
-1
32
34
31


## Iterate through elements as numpy elements

In [5]:
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

21
22
-108
31
-1
32
34
31


## Iterate through first n elements in tf dataset

In [6]:
for sales in tf_dataset.take(3):
    print(sales.numpy())

21
22
-108


### Filter sales numbers that are < 0

In [7]:
tf_dataset = tf_dataset.filter(lambda x: x>0)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

21
22
31
32
34
31


### Convert sales numbers from USA dollars ($) to Indian Rupees (INR) Assuming 1->86 conversation rate

In [8]:
tf_dataset = tf_dataset.map(lambda x: x*86)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

1806
1892
2666
2752
2924
2666


# Shuffling

In [9]:
tf_dataset = tf_dataset.shuffle(2)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

1806
2666
1892
2752
2924
2666


For more information about the `batch`, `repeat`, and `shuffle` methods in TensorFlow Dataset, refer to this [Stack Overflow discussion](https://stackoverflow.com/questions/53514495/what-does-batch-repeat-and-shuffle-do-with-tensorflow-dataset).

# Batching

In [10]:
for sales_batch in tf_dataset.batch(2):
    print(sales_batch.numpy())

[1806 2666]
[1892 2752]
[2924 2666]


### Perform all of the above operations in one shot

In [11]:
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)

tf_dataset = tf_dataset.filter(lambda x: x>0).map(lambda y: y*72).shuffle(2).batch(2)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

[1584 1512]
[2232 2304]
[2448 2232]


# Images

In [12]:
images_ds = tf.data.Dataset.list_files('/kaggle/input/image-data/images/*/*', shuffle=False)

In [13]:
image_count = len(images_ds)
image_count

130

In [14]:
type(images_ds)

tensorflow.python.data.ops.from_tensor_slices_op._TensorSliceDataset

In [15]:
for file in images_ds.take(3):
    print(file.numpy())

b'/kaggle/input/image-data/images/cat/20 Reasons Why Cats Make the Best Pets....jpg'
b'/kaggle/input/image-data/images/cat/7 Foods Your Cat Can_t Eat.jpg'
b'/kaggle/input/image-data/images/cat/A cat appears to have caught the....jpg'


In [16]:
images_ds = images_ds.shuffle(200)
for file in images_ds.take(3):
    print(file.numpy())

b'/kaggle/input/image-data/images/dog/Rescue turns dog with untreatable tumor....jpg'
b'/kaggle/input/image-data/images/dog/11 Things Humans Do That Dogs Hate.jpg'
b'/kaggle/input/image-data/images/dog/Most Popular Dog Names of 2020....jpg'


In [17]:
class_names = ["cat","dog"]

In [18]:
train_size = int(image_count*0.8)
train_ds = images_ds.take(train_size)
test_ds = images_ds.skip(train_size)

In [19]:
len(train_ds)

104

In [20]:
len(test_ds)

26

In [21]:
s = "/kaggle/input/image-data/images/cat/The Cat Health Checklist_ Everything....jpg"
s.split("/")[-2]

'cat'

In [22]:
def get_label(file_path):
    parts = tf.strings.split(file_path, os.path.sep)
    return parts[-2]

In [23]:
get_label("/kaggle/input/image-data/images/dog/The Cost of Owning a Dog.jpg")

<tf.Tensor: shape=(), dtype=string, numpy=b'dog'>

In [24]:
def process_image(file_path):
    label = get_label(file_path)
    img = tf.io.read_file(file_path) # load the raw data from the file as a string
    img = tf.image.decode_jpeg(img)
    img = tf.image.resize(img, [128, 128])
    return img, label

In [25]:
img, label = process_image("/kaggle/input/image-data/images/dog/The Cost of Owning a Dog.jpg")
# img.numpy()[:2]

In [26]:
for t in train_ds.take(4):
    print(t.numpy())

b'/kaggle/input/image-data/images/dog/Rottweiler Dog Breed Information....jpg'
b'/kaggle/input/image-data/images/dog/Most Popular Breeds \xe2\x80\x93 American Kennel Club.jpg'
b'/kaggle/input/image-data/images/dog/Mood-Boosting Benefits of Pets....jpg'
b'/kaggle/input/image-data/images/cat/7 Foods Your Cat Can_t Eat.jpg'


In [27]:
# train_ds = train_ds.map(process_image)
# test_ds = test_ds.map(process_image)

In [28]:
# train_ds = train_ds.map(process_image)
# for image, label in train_ds.take(1):
#     print("****",image)
#     print("****",label)

In [29]:
# for image, label in train_ds.take(1):
#     print("****",image)
#     print("****",label)

In [30]:
def scale(image, label):
    return image/255, label

In [31]:
# train_ds = train_ds.map(scale)

In [32]:
# for image, label in train_ds.take(5):
#     print("****Image: ",image.numpy()[0][0])
#     print("****Label: ",label.numpy())